<a href="https://colab.research.google.com/github/MaikarfiJesse/transfer_learning_assignment/blob/main/Transfer_model_Dates_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dates fruits Classification

This project is aimed to use pretrained models with a new dataset to classify different types of dates fruits from their images.

##Import libraries

In [8]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras as keras
from tensorflow.keras import regularizers
from tensorflow.keras.initializers import GlorotNormal
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import cv2
import pathlib
import matplotlib as mpl

##INSTALL DATASET


In [9]:
img_width = 448
img_height = 448
batch_size = 64
color = 3
dataset_folder = "date-fruit-image-dataset-in-controlled-environment"
dataset_url = 'https://www.kaggle.com/wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment'
classes_name = ['Ajwa', 'Galaxy', 'Medjool', 'Meneifi', 'Nabtat Ali', 'Rutab', 'Shaishe', 'Sokari', 'Sugaey']


In [10]:
!pip install opendatasets

import opendatasets as od
od.download(dataset_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: jessemaikarfi
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment


100%|██████████| 3.11G/3.11G [00:45<00:00, 74.1MB/s]


## Read the dataset

In [11]:
dataset = pathlib.Path(dataset_folder)

def directory_dataset(dataset):
  folders = []
  for i in dataset.iterdir():
    if i.is_dir():
      folders.append(i)
  return folders

folders = directory_dataset(dataset)

### Total number of Images

In [12]:
def number_of_images_in_dataset(dataset):
  images = list(dataset.glob("*/*.*"))
  return len(images)
print("number of all images in dataset: {}".format(number_of_images_in_dataset(dataset)))

number of all images in dataset: 1658


###number of Images per folder

In [13]:
def number_of_images_in_each_folder(folders):
  for i in folders:
    str_ = "{}: {}".format(i, len(list(pathlib.Path(i).glob("*.*"))))
    print(str_)

number_of_images_in_each_folder(folders)

date-fruit-image-dataset-in-controlled-environment/Galaxy: 190
date-fruit-image-dataset-in-controlled-environment/Rutab: 146
date-fruit-image-dataset-in-controlled-environment/Meneifi: 232
date-fruit-image-dataset-in-controlled-environment/Nabtat Ali: 177
date-fruit-image-dataset-in-controlled-environment/Medjool: 135
date-fruit-image-dataset-in-controlled-environment/Shaishe: 171
date-fruit-image-dataset-in-controlled-environment/Sokari: 264
date-fruit-image-dataset-in-controlled-environment/Sugaey: 168
date-fruit-image-dataset-in-controlled-environment/Ajwa: 175


###Blurring images

In [14]:
def Blurring(image):
  image = cv2.medianBlur(image,5)
  return image

def image_generator(dataset_folder):
  datagen = ImageDataGenerator( rescale=1/255, validation_split=0.1,vertical_flip=True ,horizontal_flip=True,width_shift_range=0.2,height_shift_range=0.2,
                               rotation_range = 5, shear_range = 0.02,zoom_range = 0.02, preprocessing_function = Blurring)

  train_generator = datagen.flow_from_directory(
      dataset_folder,
      target_size=(img_height, img_width),
      batch_size=batch_size,
      class_mode='categorical',
      shuffle=True,
      subset='training')

  validation_generator = datagen.flow_from_directory(
      dataset_folder,
      target_size=(img_height, img_width),
      batch_size= batch_size,
      shuffle=False,
      class_mode='categorical',
      subset='validation')

  return [train_generator, validation_generator]

In [15]:

[training_dataset, validation_dataset] = image_generator(dataset)

Found 1496 images belonging to 9 classes.
Found 162 images belonging to 9 classes.


##Pretrained Models

In [16]:
from tensorflow.keras.applications import VGG16, MobileNetV2, VGG19
from tensorflow.keras.models import Model


###Load pre-trained models


In [17]:
# Load pre-trained models
base_models = [VGG16, MobileNetV2, VGG19]
pre_models = []

for base_model in base_models:
    base = base_model(weights='imagenet', include_top=False, input_shape=(img_height, img_width, color))
    pre_models.append(base)


for pretrained_model in pre_models:
    for layer in pretrained_model.layers:
        layer.trainable = False

58889256/58889256 [==============================] - 0s 0us/step


80134624/80134624 [==============================] - 1s 0us/step


###layers modification

In [18]:
models = []
for pretrained_model in pre_models:
    last_layer = Flatten()(pretrained_model.output)
    final_layer = Dense(9, activation='softmax')(last_layer)

    model = Model(inputs=pretrained_model.input, outputs=final_layer)
    models.append(model)

###Metrics

In [19]:
me = [
      TruePositives(name='tp'),
      FalsePositives(name='fp'),
      TrueNegatives(name='tn'),
      FalseNegatives(name='fn'),
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc')
]

###Compiling

In [20]:
for model in models:
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=me)

In [21]:
call_1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=30, restore_best_weights=True)
callbacks = [call_1]

##Training

In [17]:
epochs=5
history = models[0].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

Epoch 1/5
24/24 [==============================] - 3153s 130s/step - loss: 6.7167 - tp: 233.0000 - fp: 875.0000 - tn: 11093.0000 - fn: 1263.0000 - accuracy: 0.8412 - precision: 0.2103 - recall: 0.1557 - auc: 0.5887
Epoch 2/5
24/24 [==============================] - 3107s 129s/step - loss: 1.8654 - tp: 518.0000 - fp: 430.0000 - tn: 11538.0000 - fn: 978.0000 - accuracy: 0.8954 - precision: 0.5464 - recall: 0.3463 - auc: 0.8199
Epoch 3/5
24/24 [==============================] - 3297s 137s/step - loss: 0.9083 - tp: 826.0000 - fp: 190.0000 - tn: 11778.0000 - fn: 670.0000 - accuracy: 0.9361 - precision: 0.8130 - recall: 0.5521 - auc: 0.9481
Epoch 4/5
24/24 [==============================] - 3290s 136s/step - loss: 0.6678 - tp: 999.0000 - fp: 142.0000 - tn: 11826.0000 - fn: 497.0000 - accuracy: 0.9525 - precision: 0.8755 - recall: 0.6678 - auc: 0.9741
Epoch 5/5
24/24 [==============================] - 3301s 137s/step - loss: 0.7931 - tp: 1032.0000 - fp: 235.0000 - tn: 11733.0000 - fn: 464.000

In [18]:
epochs=3
history = models[1].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

Epoch 1/3
24/24 [==============================] - 454s 18s/step - loss: 13.3019 - tp: 1736.0000 - fp: 968.0000 - tn: 22968.0000 - fn: 1256.0000 - accuracy: 0.9174 - precision: 0.6420 - recall: 0.5802 - auc: 0.8253
Epoch 2/3
24/24 [==============================] - 442s 18s/step - loss: 3.2388 - tp: 1131.0000 - fp: 361.0000 - tn: 11607.0000 - fn: 365.0000 - accuracy: 0.9461 - precision: 0.7580 - recall: 0.7560 - auc: 0.9002
Epoch 3/3
24/24 [==============================] - 425s 17s/step - loss: 2.2845 - tp: 1253.0000 - fp: 243.0000 - tn: 11725.0000 - fn: 243.0000 - accuracy: 0.9639 - precision: 0.8376 - recall: 0.8376 - auc: 0.9304


In [ ]:
epochs=3
history = models[2].fit(training_dataset, epochs=epochs, callbacks = callbacks, batch_size = batch_size
)

Epoch 1/3
 1/24 [>.............................] - ETA: 1:26:31 - loss: 1.4309 - tp: 30.0000 - fp: 23.0000 - tn: 489.0000 - fn: 34.0000 - accuracy: 0.9010 - precision: 0.5660 - recall: 0.4688 - auc: 0.8888